In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx


# =======================
# Driven Vortex Flow Field
# =======================
def vortex_velocity(x, y, t, A=8.35, B=-2.55, e=0.2):
    u = A * (1 + e * np.sin(np.pi * t)) * np.cos(y) + B * np.sin(2 * x) * np.cos(y)
    v = -2 * B * np.cos(2 * x) * np.sin(y)
    return u, v

# ==========================
# RK4 Time Integration Step
# ==========================
def rk4_step(pos, t, dt):
    x, y = pos
    u1, v1 = vortex_velocity(x, y, t)
    u2, v2 = vortex_velocity(x + 0.5*dt*u1, y + 0.5*dt*v1, t + 0.5*dt)
    u3, v3 = vortex_velocity(x + 0.5*dt*u2, y + 0.5*dt*v2, t + 0.5*dt)
    u4, v4 = vortex_velocity(x + dt*u3, y + dt*v3, t + dt)
    
    x_new = x + dt * (u1 + 2*u2 + 2*u3 + u4) / 6.0
    y_new = y + dt * (v1 + 2*v2 + 2*v3 + v4) / 6.0

    # periodic boundary conditions
    x_new = x_new % np.pi
    y_new = y_new % np.pi
    return x_new, y_new

# ==========================
# Grid Setup
# ==========================
nx_cells, ny_cells = 100, 100
x_vals = np.linspace(0, np.pi, nx_cells)
y_vals = np.linspace(0, np.pi, ny_cells)
X, Y = np.meshgrid(x_vals, y_vals)
positions = np.vstack([X.ravel(), Y.ravel()]).T
N = nx_cells * ny_cells

# ==========================
# Time Integration Parameters
# ==========================
dt = 0.05       # step size
s = 2.0         # integration time
steps = int(s / dt)
# sample many starting phases
t0_samples = np.linspace(0, 2.0, 20)  # 20 different starting phases over one forcing period

# ==========================
# Build Flow Network
# ==========================
G = nx.DiGraph()
G.add_nodes_from(range(N))

for t0 in t0_samples:
    for i, (x0, y0) in enumerate(positions):
        x, y = x0, y0
        t = t0
        for _ in range(steps):
            x, y = rk4_step((x, y), t, dt)
            t += dt
        # map back to grid cell
        xi = int((x / np.pi) * nx_cells) % nx_cells
        yi = int((y / np.pi) * ny_cells) % ny_cells
        j = yi * nx_cells + xi
        G.add_edge(i, j)

# ==========================
# Compute In/Out Degrees
# ==========================
in_degree = np.array([G.in_degree(n) for n in range(N)])
out_degree = np.array([G.out_degree(n) for n in range(N)])

in_degree_grid = in_degree.reshape((ny_cells, nx_cells))
out_degree_grid = out_degree.reshape((ny_cells, nx_cells))

# ==========================
# Plot Results
# ==========================
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

vmax = max(in_degree_grid.max(), out_degree_grid.max())
vmin = min(in_degree_grid.min(), out_degree_grid.min())

# (a) In-degree
im0 = axes[0].imshow(in_degree_grid, origin='lower', cmap='plasma', vmin=vmin, vmax=vmax)
axes[0].set_title("a", loc='left', fontsize=14)
axes[0].set_xlabel("X")
axes[0].set_ylabel("Y")
fig.colorbar(im0, ax=axes[0], label=r"$k_{in}$")

# (b) Out-degree
im1 = axes[1].imshow(out_degree_grid, origin='lower', cmap='plasma', vmin=vmin, vmax=vmax)
axes[1].set_title("b", loc='left', fontsize=14)
axes[1].set_xlabel("X")
axes[1].set_ylabel("Y")
fig.colorbar(im1, ax=axes[1], label=r"$k_{out}$")

plt.tight_layout()
plt.show()
